In [6]:
import ollama
import numpy as np 
import pickle

file_path=r'C:\Users\ashli\Documents\Generative AI\galaxy.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text=f.read()
chunk_size = 500
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

chunks    



["A galaxy is a system of stars, stellar remnants, interstellar gas, dust, and dark matter bound together by gravity.The word is derived from the Greek galaxias, literally 'milky', a reference to the Milky Way galaxy that contains the Solar System. Galaxies, averaging an estimated 100 million stars,[3] range in size from dwarfs with less than a thousand stars,[4] to the largest galaxies known – supergiants with one hundred trillion stars, each orbiting its galaxy's centre of mass. Most of the mass",
 ' in a typical galaxy is in the form of dark matter, with only a few percent of that mass visible in the form of stars and nebulae. Supermassive black holes are a common feature at the centres of galaxies.\n\nGalaxies are categorised according to their visual morphology as elliptical, spiral, or irregular. The Milky Way is an example of a spiral galaxy. In addition to shape, galaxies may be notable due to special properties, such as interacting with another galaxy, producing stars at an un

In [9]:
embeddings=[]
for chunk in chunks:
    response = ollama.embeddings(model='nomic-embed-text',prompt=chunk)
    embeddings.append(response["embedding"])
embeddings    

[[0.07397955656051636,
  1.6253893375396729,
  -2.997431755065918,
  -1.291530966758728,
  0.7261798977851868,
  0.7807971239089966,
  -0.792414665222168,
  -0.14922557771205902,
  -0.004440614953637123,
  -1.1836096048355103,
  0.825027585029602,
  0.7332260608673096,
  1.6633985042572021,
  0.9332305192947388,
  1.231292486190796,
  -1.13820219039917,
  -0.8035904169082642,
  -0.1759367436170578,
  -0.11280703544616699,
  1.2603998184204102,
  -1.269950270652771,
  -0.797013521194458,
  0.4406906068325043,
  -1.510778546333313,
  0.7559134364128113,
  1.4921178817749023,
  0.09417260438203812,
  -1.0050944089889526,
  -0.03875607252120972,
  -0.9121679067611694,
  0.7136138081550598,
  -0.387918621301651,
  -0.18032872676849365,
  0.31316041946411133,
  -0.6158618330955505,
  -1.028671383857727,
  0.29313984513282776,
  0.8143345713615417,
  1.1043140888214111,
  0.2051728218793869,
  0.17463576793670654,
  0.13024307787418365,
  0.4559265077114105,
  0.12718121707439423,
  0.8452310

In [12]:
data_to_save = {
    "chunks": chunks,
    "embeddings": embeddings
}


In [13]:
data_to_save.values()

dict_values([["A galaxy is a system of stars, stellar remnants, interstellar gas, dust, and dark matter bound together by gravity.The word is derived from the Greek galaxias, literally 'milky', a reference to the Milky Way galaxy that contains the Solar System. Galaxies, averaging an estimated 100 million stars,[3] range in size from dwarfs with less than a thousand stars,[4] to the largest galaxies known – supergiants with one hundred trillion stars, each orbiting its galaxy's centre of mass. Most of the mass", ' in a typical galaxy is in the form of dark matter, with only a few percent of that mass visible in the form of stars and nebulae. Supermassive black holes are a common feature at the centres of galaxies.\n\nGalaxies are categorised according to their visual morphology as elliptical, spiral, or irregular. The Milky Way is an example of a spiral galaxy. In addition to shape, galaxies may be notable due to special properties, such as interacting with another galaxy, producing st

In [14]:
output_pickle ='knowledge.pkl'
with open(output_pickle,'wb') as f:
    pickle.dump(data_to_save,f)

In [15]:
def cosine_similarity(v1,v2):
    return np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [20]:
k =2 
pickle_file=r"C:\Users\ashli\Documents\Generative AI\knowledge.pkl"

with open(pickle_file,'rb') as f:
    data=pickle.load(f)

chunks = data["chunks"]
doc_embeddings=data["embeddings"]

user_query = input("Enter the query: ")

response = ollama.embeddings(
    model='nomic-embed-text',
    prompt=user_query
)

query_embedding = response['embedding']

similarities = [cosine_similarity(query_embedding,emb) for emb in doc_embeddings]
top_k_indices=np.argsort(similarities)[-k:][::-1]

retrieved_chunks=[chunks[i] for i in top_k_indices]
combined_context= "\n---\n".join(retrieved_chunks)

prompt = f"""Using the following context composed of {k} chunks:

{combined_context}

Question: {user_query}
Answer:"""

output = ollama.generate(
    model="gemma3:1b",
    prompt=prompt
)


output['response']




'Spiral galaxy is a type of galaxy characterized by its shape, which is often visually noticeable due to interacting with other galaxies.'